# RUN

Whenever a new year's data has to be added, say for AY 2023, we will be performing DW corrections on 2022 data and 2021 data. So exporting of data will be done for AY 2021, AY 2022 and AY 2023

In [ ]:
!pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 95.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from glob import glob
import re
import matplotlib.pyplot as plt
import json
import numpy as np
from scipy import stats as st
import ee
import shapely.geometry
from shapely.geometry import Point, Polygon
import geopandas as gpd
import fiona
from math import sqrt
from shapely import wkt
import os
import time
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-mtpictd')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
year = 2022

In [ ]:
# Choose appropriate ACZ

# agroclimatic_zone = "Eastern Plateau & Hills Region"
agroclimatic_zone = "Southern Plateau and Hills Region"
# agroclimatic_zone = "East Coast Plains & Hills Region"
# agroclimatic_zone = "Western Plateau and Hills Region"
# agroclimatic_zone = "Central Plateau & Hills Region"
# agroclimatic_zone = "Lower Gangetic Plain Region"
# agroclimatic_zone = "Middle Gangetic Plain Region"
# agroclimatic_zone = "Eastern Himalayan Region"
# agroclimatic_zone = "Western Himalayan Region"
# agroclimatic_zone = "Upper Gangetic Plain Region"
# agroclimatic_zone = "Trans Gangetic Plain Region"

# agroclimatic_zone = "West Coast Plains & Ghat Region"
# agroclimatic_zone = "Gujarat Plains & Hills Region"
# agroclimatic_zone = "Western Dry Region"

In [ ]:
agroclimaticZone_acronym_dict = {'Eastern Plateau & Hills Region': 'EPAHR',
                               'Southern Plateau & Hills Region': 'SPAHR',
                               'East Coast Plains & Hills Region': 'ECPHR',
                               'Western Plateau & Hills Region': 'WPAHR',
                               'Central Plateau & Hills Region': 'CPAHR',
                               'Lower Gangetic Plain Region': 'LGPR',
                                'Middle Gangetic Plain Region': 'MGPR',
                                'Eastern Himalayan Region': 'EHR',
                                'Western Himalayan Region': 'WHR',
                                'Upper Gangetic Plain Region': 'UGPR',
                                'Trans Gangetic Plain Region': 'TGPR',
                                'West Coast Plains & Ghat Region': 'WCPGR',
                                'Gujarat Plains & Hills Region': 'GPHR',
                                'Western Dry Region': 'WDR'}

In [ ]:
india_boundary = ee.FeatureCollection("projects/ee-mtpictd/assets/harsh/Agroclimatic_regions")
agrozone = india_boundary.filter(ee.Filter.eq('regionname', agroclimatic_zone)).geometry()
india_district_boundary = ee.FeatureCollection("projects/ee-indiasat/assets/india_district_boundaries")

In [ ]:
s1_bands = ['VV', 'VH', 'angle']
s2_bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B10', 'B11','B12']

In [ ]:
START_DATE = {year-2: {'kharif': str(year-2)+'-07-01', 'rabi': str(year-2)+'-11-01', 'zaid': str(year-1)+'-03-01'},
              year-1: {'kharif': str(year-1)+'-07-01', 'rabi': str(year-1)+'-11-01', 'zaid': str(year)+'-03-01'},
              year: {'kharif': str(year)+'-07-01', 'rabi': str(year)+'-11-01', 'zaid': str(year+1)+'-03-01'}}

END_DATE = {year-2: {'kharif': str(year-2)+'-10-31', 'rabi': str(year-1)+'-02-28', 'zaid': str(year-1)+'-06-30'},
            year-1: {'kharif': str(year-1)+'-10-31', 'rabi': str(year)+'-02-28', 'zaid': str(year)+'-06-30'},
            year: {'kharif': str(year)+'-10-31', 'rabi': str(year+1)+'-02-28', 'zaid': str(year+1)+'-06-30'}}

In [ ]:
print(START_DATE)
print(END_DATE)

{2020: {'kharif': '2020-07-01', 'rabi': '2020-11-01', 'zaid': '2021-03-01'}, 2021: {'kharif': '2021-07-01', 'rabi': '2021-11-01', 'zaid': '2022-03-01'}, 2022: {'kharif': '2022-07-01', 'rabi': '2022-11-01', 'zaid': '2023-03-01'}}
{2020: {'kharif': '2020-10-31', 'rabi': '2021-02-28', 'zaid': '2021-06-30'}, 2021: {'kharif': '2021-10-31', 'rabi': '2022-02-28', 'zaid': '2022-06-30'}, 2022: {'kharif': '2022-10-31', 'rabi': '2023-02-28', 'zaid': '2023-06-30'}}


In [ ]:
# Will take an AOI geometry as input and return the 10km x 10km grids in it as a list
def createGrids(aoi):
    proj = ee.Projection('EPSG:4326')
    gridSize = 10000
    grid = aoi.coveringGrid(proj, gridSize)
    features = grid.getInfo()['features']
    return features

In [ ]:
###################### Pixel-based cloud masking ###############

def get_s2_image(aoi, start_date, end_date):
    # First, get S2 imagery collection
    s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(
        start_date, end_date).filterBounds(aoi)

    # Get cloud probability collection for the same area and time period
    cloud_prob_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate(
        start_date, end_date).filterBounds(aoi)

    # Join the two collections
    joined = ee.Join.saveFirst('cloud_prob').apply(
        primary=s2_col,
        secondary=cloud_prob_col,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )

    # Apply cloud masking to each image
    def apply_cloud_mask(img):
        # Get the cloud probability image
        cloud_prob = ee.Image(img.get('cloud_prob'))

        # If cloud probability data exists, use it along with QA60
        cloud_mask = ee.Algorithms.If(
            ee.Algorithms.IsEqual(cloud_prob, None),
            # If no cloud probability data, use only QA60
            img.select('QA60').bitwiseAnd(1 << 10).eq(0).And(
                img.select('QA60').bitwiseAnd(1 << 11).eq(0)),
            # If cloud probability data exists, use both
            img.select('QA60').bitwiseAnd(1 << 10).eq(0).And(
                img.select('QA60').bitwiseAnd(1 << 11).eq(0)).And(
                cloud_prob.select('probability').lt(20))  # Cloud probability less than 50%
        )

        return img.updateMask(ee.Image(cloud_mask)).divide(10000)

    # Apply the cloud masking to the collection and select bands
    masked_collection = ee.ImageCollection(joined).map(apply_cloud_mask).select(s2_bands)

    # Create a median composite
    return masked_collection.median().clip(aoi)

def get_s1_image(aoi, start_date, end_date):
  # s1_bands_season = [band + '_' + season for band in s1_bands]
  return ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(start_date , end_date).filterBounds(aoi).filter(
      ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(
          ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(
              ee.Filter.eq('instrumentMode', 'IW')).select(s1_bands).median().clip(aoi)

In [ ]:
def save_data_csv(data_points, img_name, district, year):
  print("Saving data for", district, year)
  new_img_name = img_name.replace('&', 'and')
  new_img_name = new_img_name.replace('(', '')
  new_img_name = new_img_name.replace(')', '')
  task = ee.batch.Export.table.toDrive(
      collection = data_points,
      description = new_img_name,
      folder = f"{agroclimaticZone_acronym_dict[agroclimatic_zone]}_{district}_{year}",
      fileNamePrefix = new_img_name,
      fileFormat = 'CSV'
      )
  task.start()
  return task

In [ ]:
df = pd.read_csv(f'drive/MyDrive/harsh/{agroclimatic_zone}.csv')
dist_list = list(df['Name'])

In [ ]:
years = [str(year-2), str(year-1), str(year)]
year_0 = years[0]
year_1 = years[1]
year_2 = years[2]
year_suffix = {year_0: year_0[-2:], year_1: year_1[-2:], year_2: year_2[-2:]}

In [ ]:
def get_dw_tree_cover(aoi, start_date, end_date, scale = 25):
    tree_cover_dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate(start_date, end_date) \
                  .filterBounds(aoi).select('label').mode().clip(aoi)
    return tree_cover_dw.updateMask(tree_cover_dw.eq(1)).reproject(crs='EPSG:4326', scale=scale)

def get_is_tree_cover(aoi, year, scale = 25):
    year = int(year)
    indiasat_asset = f"projects/ee-corestackdev/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_{year}_{year+1}"
    lulc_image = ee.Image(indiasat_asset).select("predicted_label").clip(aoi)
    return lulc_image.updateMask(lulc_image.eq(6)).reproject(crs='EPSG:4326', scale=scale)

def get_tree_cover(aoi, year, scale = 25):
    year = int(year)
    start_date = ee.Date(f'{year}-07-01')
    end_date = ee.Date(f'{year+1}-06-30')

    tree_cover_is = get_is_tree_cover(aoi, year, scale)
    tree_cover_dw = get_dw_tree_cover(aoi, start_date, end_date, scale)
    tree_cover = tree_cover_is.mask().Or(tree_cover_dw.mask())
    tree_cover = tree_cover.updateMask(tree_cover)
    return tree_cover.reproject(crs='EPSG:4326', scale=scale)

In [ ]:
for year in years:

  total_time = 0

  dist_cnt = 0
  for district in dist_list:

      # if dist_cnt < 3:
      #     dist_cnt += 1
      #     continue

      start_time = time.time()

      district_aoi = india_district_boundary.filter(ee.Filter.eq('Name', district)).geometry()
      district_aoi = district_aoi.intersection(agrozone)
      features = createGrids(district_aoi)

      print(f'Year {year}, District {dist_cnt}: {district}, grids: {len(features)}')

      path = f'drive/MyDrive/{agroclimatic_zone}/{district}/{year}/'
      os.makedirs(path, exist_ok=True)

      df = pd.DataFrame()
      df['grid_num'] = list(range(len(features)))
      tree_points_list = []

      i = 0
      for feature in features:
          print(f'Grid {i}')

          coord = feature['geometry']['coordinates'][0]
          aoi = ee.Geometry.Polygon(coord)
          aoi = aoi.intersection(district_aoi)

          img_name = district + "_" + str(i) + "_" + str(year)

          start_date = START_DATE[year]
          end_date = END_DATE[year]

          # Get tree cover using both Dynamic World and IndiaSat (union)
          tree_cover = get_tree_cover(aoi, year, scale=25)

          season = 'kharif'
          try:
              image = get_s1_image(aoi, start_date[season], end_date[season])
              image = image.updateMask(tree_cover)
          except Exception as exp:
              print("S1 Error occured: ", season, exp)

          try:
              s2_data = get_s2_image(aoi, start_date[season], end_date[season])
              s2_data = s2_data.updateMask(tree_cover)
              image = image.addBands(s2_data).select(s1_bands + s2_bands)
              image = image.rename([band + '_kharif' for band in s1_bands + s2_bands])
          except Exception as exp:
              print("S2 Error occured: ", season, exp)

          for season in ['rabi', 'zaid']:
              try:
                  s1_data = get_s1_image(aoi, start_date[season], end_date[season])
                  s1_data = s1_data.updateMask(tree_cover)

              except Exception as exp:
                  print("S1 Error occured: ", season, exp)

              try:
                  s2_data = get_s2_image(aoi, start_date[season], end_date[season])
                  s2_data = s2_data.updateMask(tree_cover)
                  image_merged = s1_data.addBands(s2_data).select(s1_bands + s2_bands)
                  image_merged = image_merged.rename([band + '_' + season for band in s1_bands + s2_bands])
                  image = image.addBands(image_merged)

              except Exception as exp:
                  print("S2 Error occured: ", season, exp)

          sample_points = image.sample(
                  region = aoi,
                  scale = 25,
                  factor = 1,
                  tileScale = 10,
                  geometries = True
              )

          sample_tree_cover = tree_cover.sample(
              region = aoi,
              scale = 25,
              factor = 1,
              tileScale = 10,
              geometries = True
          )

          try:
              tree_points_list.append(sample_tree_cover.size().getInfo())
          except:
              tree_points_list.append(0)

          try:
              task = save_data_csv(sample_points, img_name, district, year)
              prev_task = task
              # tasks[year][district] = task

          except Exception as e:
              print(e)
              while prev_task.status()['state'] != 'COMPLETED' and prev_task.status()['state'] != 'FAILED':
                  # time.sleep(10)
                  continue
              task = save_data_csv(sample_points, img_name, path, year)
              # tasks[year][district] = task
              prev_task = task

          i += 1

      df['tree_points'] = tree_points_list
      df.to_csv(path + 'tree_cover_points.csv', index=False)

      dist_cnt += 1
      end_time = time.time()

      total_time += (end_time - start_time)
      print(total_time)

  print("Waiting for last task to be completed...")
  while prev_task.status()['state'] != 'COMPLETED' and prev_task.status()['state'] != 'FAILED':
      continue
  print("Last task completed!")

  total_time += (time.time() - end_time)
  print("Total Time Taken:", total_time)


Year 2022, District 0: Custom_KML_Area, grids: 8
Grid 0
Saving data for Custom_KML_Area 2022
Grid 1
Saving data for Custom_KML_Area 2022
Grid 2
Saving data for Custom_KML_Area 2022
Grid 3
Saving data for Custom_KML_Area 2022
Grid 4
Saving data for Custom_KML_Area 2022
Grid 5
Saving data for Custom_KML_Area 2022
Grid 6
Saving data for Custom_KML_Area 2022
Grid 7
Saving data for Custom_KML_Area 2022
17.544011116027832
Waiting for last task to be completed...
